In [ ]:
!pip install mp_api
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mp_api.client import MPRester

# Set your API key (replace with your actual key)
API_KEY = "3ZVUGDS6a2pAXCs7cSuwBRXMuHni8qLQ"

def get_filtered_band_gaps(min_gap, max_gap):
    """Retrieve materials and extract band gaps within the specified range."""
    with MPRester(API_KEY) as mpr:
        materials = mpr.materials.search(fields=["material_id", "formula_pretty", "entries"])

    if not materials:
        print("No materials found for the given query.")
        return pd.DataFrame()

    # Extract band gap from 'entries'
    filtered_data = []
    for mat in materials:
        for entry in getattr(mat, "entries", []):  # Correct way to access entries
            if hasattr(entry, "band_gap"):  # Ensure 'band_gap' exists before accessing
                band_gap = entry.band_gap
                if band_gap is not None and min_gap <= band_gap <= max_gap:
                    filtered_data.append({"Formula": mat.formula_pretty, "Band Gap (eV)": band_gap})

    return pd.DataFrame(filtered_data)

def visualize_band_gaps(df):
    """Plot histogram of filtered band gaps."""
    if df.empty:
        print("No data available for visualization.")
        return

    plt.figure(figsize=(8, 5))
    plt.hist(df["Band Gap (eV)"], bins=20, color="green", alpha=0.7)
    plt.xlabel("Band Gap (eV)")
    plt.ylabel("Frequency")
    plt.title("Distribution of Band Gaps in Selected Range")
    plt.show()

# Get user input before function execution
min_gap = float(input("Enter minimum band gap (eV): "))
max_gap = float(input("Enter maximum band gap (eV): "))

# Retrieve and visualize results
df_filtered = get_filtered_band_gaps(min_gap, max_gap)

if not df_filtered.empty:
    visualize_band_gaps(df_filtered)
    print(df_filtered.head())
else:
    print("No materials match the specified band gap range.")